In [8]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, Model
from music21 import *
import os
import IPython.display as display
import pygame
import tarfile
import pandas as pd

In [9]:
# Grab midi files from path
midi_files = [path + str(f) for f in os.listdir(path)]

In [10]:
# Play a midi file using pygame
pygame.init()
pygame.mixer.music.load(midi_files[0])
pygame.mixer.music.play()

In [11]:
# Execute this cell to stop the music
pygame.mixer.music.stop()

In [12]:
# Chordify files to form chords from multiple staves
midi_files = [converter.parse(mf).chordify() for mf in midi_files]

In [13]:
def map_to_int(chords, durations, offsets):
    unique_chords = np.unique([i for s in chords for i in s])
    mapped_chords = {c:i for i, c in enumerate(unique_chords)}
    
    unique_durations = np.unique([i for s in durations for i in s])
    mapped_durations = {d:i for i, d in enumerate(unique_durations)}
    
    unique_offsets = np.unique([i for s in offsets for i in s])
    mapped_offsets = {o:i for i, o in enumerate(unique_offsets)}
    
    print(f'Number of unique notes and chords: {len(mapped_chords)}.\n'
            f'Number of unique durations: {len(mapped_durations)}.\n'
            f'Number of unique offsets: {len(mapped_offsets)}')
    
    return mapped_chords, mapped_durations, mapped_offsets

In [14]:
def preprocess(midi_files):
    # A list of all inputs for each song
    chords = [[] for _ in range(len(midi_files))]
    durations = [[] for _ in range(len(midi_files))]
    offsets = [[] for _ in range(len(midi_files))]
    
    for i, piece in enumerate(midi_files):
        for element in piece:
            if isinstance(element, note.Note):
                chords[i].append(element.pitch)
                durations[i].append(element.duration.quarterLength)
                offsets[i].append(element.offset)
            elif isinstance(element, chord.Chord):
                chords[i].append('.'.join(str(n) for n in element.pitches))
                durations[i].append(element.duration.quarterLength)
                offsets[i].append(element.offset)
                
    chord_to_int, duration_to_int, offset_to_int = map_to_int(chords, durations, offsets)
    
    return (chords, chord_to_int), (durations, duration_to_int), (offsets, offset_to_int)

In [15]:
# Preprocess to put unique chords and durations to integers, chords mapped to integers dict[chord] = int
chords, durations, offsets = preprocess(midi_files)

Number of unique notes and chords: 4569.
Number of unique durations: 26.
Number of unique offsets: 6832


In [62]:
def train_test_split(midi_files, chords, durations, offsets, sequence=32):
    chord, chord_ix = chords
    duration, duration_ix = durations
    offset, offset_ix = offsets
    
    X_chords, Y_chords = [], []
    X_durations, Y_durations = [], []
    X_offsets, Y_offsets = [], []
    
    for s in range(len(midi_files)):
        chords_list = [chord_ix[c] for c in chord[s]]
        durations_list = [duration_ix[c] for c in duration[s]]
        offsets_list = [offset_ix[c] for c in offset[s]]
        
        for i in range(len(chords_list) - sequence):
            X_chords.append(chords_list[i:i+sequence])
            Y_chords.append(chords_list[i+1:i+sequence+1])
            
            X_durations.append(durations_list[i:i+sequence])
            Y_durations.append(durations_list[i+1:i+sequence+1])
            
            X_offsets.append(offsets_list[i:i+sequence])
            Y_offsets.append(offsets_list[i+1:i+sequence+1])
    
    return np.array(X_chords), np.array(Y_chords), np.array(X_durations), np.array(Y_durations), np.array(X_offsets), np.array(Y_offsets)

In [63]:
sequence = 32
X_chords, Y_chords, X_durations, Y_durations, X_offsets, Y_offsets = train_test_split(midi_files, chords, durations, offsets, sequence=sequence)

In [64]:
# Explore the shapes
X_chords.shape, X_durations.shape, X_offsets.shape  # shapes have 32 chords in a sequence with 11408 samples

((11408, 32), (11408, 32), (11408, 32))

In [66]:
Y_chords = Y_chords.reshape((-1, 96, 32))
Y_chords.shape

ValueError: cannot reshape array of size 365056 into shape (96,32)

In [55]:
m = X_chords.shape[0]
n_c = X_chords.shape[1]
n_d = X_durations.shape[1]
n_o = X_offsets.shape[1]

# Input dimensions for embedding
input_dim = m * sequence

# Embedding layer dimensions
embed_dim = 64

In [78]:
def lstm_model(embed_dim, n_c, n_d, n_o, sequence):
    # Define three inputs to process
    chord_input = keras.Input(shape=(None,))
    duration_input = keras.Input(shape=(None,))
    offset_input = keras.Input(shape=(None,))
    
    # Define embedding layers
    chord_embedding = layers.Embedding(n_c, embed_dim, input_length=sequence)(chord_input)
    duration_embedding = layers.Embedding(n_d, embed_dim, input_length=sequence)(duration_input)
    offset_embedding = layers.Embedding(n_o, embed_dim, input_length=sequence)(offset_input)
    
    # Concatenate together embeddings
    merge_layer = layers.Concatenate(axis=1)([chord_embedding, duration_embedding, offset_embedding])
    
    # Define an LSTM Layer
    lstm_layer = layers.LSTM(512, return_sequences=True)(merge_layer)
    
    # Define a dense layer
    dense_layer = layers.Dense(256)(lstm_layer)
    
    # Define output dense layers with softmax activation
    chord_output = layers.Dense(n_c, activation='softmax')(dense_layer)
    duration_output = layers.Dense(n_d, activation='softmax')(dense_layer)
    offset_output = layers.Dense(n_o, activation='softmax')(dense_layer)
    
    # Define a functional API model
    lstm = Model(inputs=[chord_input, duration_input, offset_input], outputs=[chord_output, duration_output, offset_output])
    
    return lstm

In [80]:
lstm = lstm_model(embed_dim, n_c, n_d, n_o, sequence)

In [81]:
# Compile model with categorical_crossentropy loss and rmsprop optimizer
lstm.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [82]:
# Define a model checkpoint callback
checkpoint_path = "../checkpoints/mix/"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                            save_weights_only=True,
                                                            monitor='val_acc',
                                                            mode='max',
                                                            save_best_only=True)

In [83]:
lstm.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, None, 64)     2048        input_25[0][0]                   
____________________________________________________________________________________________

In [70]:
lstm.fit([X_chords, X_durations, X_offsets], [Y_chords, Y_durations, Y_offsets], epochs=500, batch_size=64, callbacks=[model_checkpoint_callback])

Epoch 1/500


ValueError: in user code:

    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\keras\engine\training.py:532 train_step  **
        loss = self.compiled_loss(
    C:\Users\timothy\Anaconda3\envs\ml\lib\site-packages\tensorflow\python\keras\engine\comp